## Rentabilidades de los Fondos de Inversión Colectiva (FIC)

La rentabilidad de los Fondos de Inversión Colectiva (en adelante FIC) y Fondos de Capital Privado (en adelante FCP) está determinada por la variación del valor de unidad diaria de las unidades de inversión llamadas "Participaciones".

**URL Datos Abiertos:** https://www.datos.gov.co/Hacienda-y-Cr-dito-P-blico/Rentabilidades-de-los-Fondos-de-Inversi-n-Colectiv/qhpu-8ixx


**url_base:** https://www.datos.gov.co/resource/qhpu-8ixx.json

In [ ]:
# - Realizamos las configuraciones necesarias para conectarnos a Google Drive

In [108]:
# -  Cargamos los datos a un df
query = f'https://www.datos.gov.co/resource/qhpu-8ixx.json?$where=fecha_corte between \'{fecha_min}\' and \'{fecha_max}\'&$limit=10'
headers = {'X-App-Token':os.environ.get('X_APP_TOKEN')}
response = requests.get(query, headers=headers)
response.status_code
data = response.json()
df = pd.DataFrame(data)
df.shape

(10, 26)

In [109]:
# - Cargamos la hoja de calculo de Google Sheet y consultamos sus hojas
gc = gspread.service_account(filename='storage/keys/access-key-google-drive.json')
sheet = gc.open('FIC_dataset')
print(sheet.worksheets())

[<Worksheet 'summary' id:0>, <Worksheet 'data' id:1555986092>]


In [112]:
# - Cargamos la hoja data
sh_data = sheet.worksheet('data')

# - Enviamos los datos del df a la hoja data
sh_data.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1flfqQEOwde0c0jCgLnfSk4te_oWN-com-7LPvVe4NOo',
 'updatedRange': 'data!A1:Z11',
 'updatedRows': 11,
 'updatedColumns': 26,
 'updatedCells': 286}

In [118]:

sh_data.row_count

1000

In [119]:
https://developers.google.com/sheets/api/guides/concepts#a1_notation
    
https://gspread.readthedocs.io/en/latest/api.html#gspread.models.Worksheet.update
    
    https://www.youtube.com/watch?v=I5AihV-__0o
    

SyntaxError: invalid syntax (<ipython-input-119-12e40836f359>, line 1)

Notas:

http://veekaybee.github.io/2020/02/25/secrets/

https://github.com/trallard
https://docs.microsoft.com/en-us/windows/python/web-frameworks
https://medium.com/@tonybolanyo/variables-de-entorno-con-dotenv-en-python-4697540bf7e4

https://dev.socrata.com/foundry/www.datos.gov.co/qhpu-8ixx
https://dev.socrata.com/docs/queries/
https://dev.socrata.com/consumers/getting-started.html

https://dev.socrata.com/blog/2014/1/04/data-visualization-with-python.html


In [ ]:
# - Instalar paquetes en jupyter mac
# - !/usr/local/Cellar/jupyterlab/3.0.12/libexec/bin/python3.9 -m pip install pandas
# - !/usr/local/Cellar/jupyterlab/3.0.12/libexec/bin/python3.9 -m pip install python-dotenv
# - !/usr/local/Cellar/jupyterlab/3.0.12/libexec/bin/python3.9 -m pip install gspread

# - !/usr/local/Cellar/jupyterlab/3.0.12/libexec/bin/python3.9 -m pip show

In [100]:
import gspread

gc = gspread.service_account(filename='storage/keys/access-key-google-drive.json')
sheet = gc.open('FIC_dataset')
print(sheet.worksheets())


[<Worksheet 'summary' id:0>, <Worksheet 'data' id:1555986092>]


In [103]:
# - cargamos las hojas de la hoja de calculo
sh_summary = sheet.worksheet("summary")
sh_data = sheet.worksheet("data")


df_sh_summary = pd.DataFrame(sh_summary.get_all_records())
df_sh_summary

,label,value
0,updateDate,2016-01-01T00:00:00.000
1,minDate,2016-01-01T00:00:00.000
2,maxDate,2016-01-01T00:00:00.000
3,count,0


In [105]:
sh_data.update([df_sh_summary.columns.values.tolist()] + df_sh_summary.values.tolist())

{'spreadsheetId': '1flfqQEOwde0c0jCgLnfSk4te_oWN-com-7LPvVe4NOo',
 'updatedRange': 'data!A1:B5',
 'updatedRows': 5,
 'updatedColumns': 2,
 'updatedCells': 10}

In [106]:
df_sh_summary.columns.values.tolist()

['label', 'value']

In [107]:
df_sh_summary.values.tolist()


[['updateDate', '2016-01-01T00:00:00.000'],
 ['minDate', '2016-01-01T00:00:00.000'],
 ['maxDate', '2016-01-01T00:00:00.000'],
 ['count', 0]]

In [14]:
import requests
import pandas as pd
from dotenv import find_dotenv, load_dotenv
import logging
import os
import json
import gspread


# - Cargamos variables de entorno con secretos
#logging.getLogger().setLevel(logging.INFO)
try:
    dotenv_path = find_dotenv('storage/keys/.env')
    print('Dotenv localizado y actualizado!!!')
    #logging.info('Dotenv localizado!!!')
    load_dotenv(dotenv_path)

except:
    print('No hay variables locales')
    #logging.info('No hay variables locales')
    
# - Consultamos fecha mínima de datos en API
query = 'https://www.datos.gov.co/resource/qhpu-8ixx.json?$select=min(fecha_corte)'
response = requests.get(query)
data = response.json()
fecha_min = data[0].get('min_fecha_corte')

# - Consultamos fecha mínima de datos en API
query = 'https://www.datos.gov.co/resource/qhpu-8ixx.json?$select=max(fecha_corte)'
response = requests.get(query)
data = response.json()
fecha_max = data[0].get('max_fecha_corte')


Dotenv localizado y actualizado!!!


In [74]:
#Creamos diccionario con datos maestros
import json

dataConfig = {'fecha_min': fecha_min,
              'fecha_max': fecha_max}

with open('dataConfig.json', 'w') as file:
    json.dump(dataConfig, file, indent=4)


In [73]:
dataConfig

{'fecha_min': '2016-01-01T00:00:00.000',
 'fecha_max': '2021-03-24T00:00:00.000'}

In [ ]:
dataConfig.

In [69]:
# - Carga de dataframe con (1080237, 26)

query = f'https://www.datos.gov.co/resource/qhpu-8ixx.json?$where=fecha_corte between \'{fecha_min}\' and \'{fecha_max}\'&$limit=10000000'
headers = {'X-App-Token':os.environ.get('X_APP_TOKEN')}
response = requests.get(query, headers=headers)
response.status_code
data = response.json()
df = pd.DataFrame(data)
df.shape

(1080237, 26)

In [70]:
df.shape

(1080237, 26)

In [71]:
df.to_csv(r'Name.csv', index = False)


In [ ]:
https://www.datos.gov.co/resource/32sa-8pi3.json?%24limit=1&%24order=vigenciahasta+DESC

In [ ]:
 https://soda.demo.socrata.com/resource/4tka-6guv.json?$order=magnitude DESC
 

In [3]:
# - Conexión a datos abiertos
url = 'https://www.datos.gov.co/resource/qhpu-8ixx.json'
headers = {'X-App-Token':os.environ.get('X_APP_TOKEN')}

response = requests.get(url,headers=headers)
response.status_code

200

In [ ]:
del os.environ["X_APP_TOKEN"]


In [28]:
import requests
import pandas as pd

host_FIC = 'https://www.datos.gov.co/resource/qhpu-8ixx.json?fecha_corte=2016-01-01T00:00:00.000'

response = requests.get(host_FIC)
dataList = response.json()

df = pd.DataFrame(dataList)
df.shape

(471, 26)

In [9]:
df.head()

,fecha_corte,tipo_entidad,nombre_tipo_entidad,codigo_entidad,nombre_entidad,tipo_negocio,nombre_tipo_patrimonio,subtipo_negocio,nombre_subtipo_patrimonio,codigo_negocio,...,valor_unidad_operaciones,aportes_recibidos,retiros_redenciones,anulaciones,valor_fondo_cierre_dia_t,numero_inversionistas,rentabilidad_diaria,rentabilidad_mensual,rentabilidad_semestral,rentabilidad_anual
0,2021-03-11T00:00:00.000,5,SF-SOCIEDAD FIDUCIARIA,62,Fiduciaria Coomeva S.A.,9,FONDOS DE INVERSIÓN COLECTIVA,1,FIC DE TIPO GENERAL,82379,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2021-03-11T00:00:00.000,5,SF-SOCIEDAD FIDUCIARIA,3,Bbva Asset Management S.A. Sociedad Fiduciaria,9,FONDOS DE INVERSIÓN COLECTIVA,1,FIC DE TIPO GENERAL,8686,...,24302.192417,0.590000,0.000000,0.000000,30144702392.100000,22.000000,28.784935,-5.571943,2.069361,6.126540
2,2021-03-11T00:00:00.000,5,SF-SOCIEDAD FIDUCIARIA,3,Bbva Asset Management S.A. Sociedad Fiduciaria,9,FONDOS DE INVERSIÓN COLECTIVA,1,FIC DE TIPO GENERAL,8686,...,24132.311896,33594003.230000,572823767.900000,0.000000,206727150316.110000,2326.000000,27.897297,-6.223332,1.365411,5.394686
3,2021-03-11T00:00:00.000,5,SF-SOCIEDAD FIDUCIARIA,3,Bbva Asset Management S.A. Sociedad Fiduciaria,9,FONDOS DE INVERSIÓN COLECTIVA,1,FIC DE TIPO GENERAL,8734,...,2636.563207,1867437001.740000,5085042766.000000,0.000000,47833818074.770000,19.000000,3.925802,0.354870,1.640968,3.504053
4,2021-03-11T00:00:00.000,5,SF-SOCIEDAD FIDUCIARIA,3,Bbva Asset Management S.A. Sociedad Fiduciaria,9,FONDOS DE INVERSIÓN COLECTIVA,1,FIC DE TIPO GENERAL,8734,...,2566.733371,1541482230.790000,1554231194.950000,0.000000,331460068654.590000,8861.000000,3.515523,-0.041349,1.239687,3.095437
